# L4: Optimize DSPy Agent with DSPy Optimizer

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]  = os.getenv("OPENAI_API_KEY")

In [4]:
import mlflow

In [5]:
import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.    # the format for that file is (without the comment)                      #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService                                                            
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key

def get_mlflow_tracking_uri():
    load_env()
    # Try to get from environment, fallback to default
    dlai_url = os.environ.get('DLAI_LOCAL_URL', 'http://localhost:{port}')
    return dlai_url.format(port=8080)

mlflow_tracking_uri = get_mlflow_tracking_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [6]:
mlflow.set_experiment("dspy_course_4")

2025/06/14 11:10:52 INFO mlflow.tracking.fluent: Experiment with name 'dspy_course_4' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/322957703646966226', creation_time=1749874252307, experiment_id='322957703646966226', last_update_time=1749874252307, lifecycle_stage='active', name='dspy_course_4', tags={}>

In [9]:
mlflow.dspy.autolog(log_evals=True, log_compiles=True, log_traces_from_compile=True)

In [10]:
import dspy

dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Build a RAG Agent

In [23]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

In [25]:
import json

# Load trainset
trainset = []
with open("trainset.jsonl", "r") as f:
    for line in f:
        trainset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

# Load valset
valset = []
with open("valset.jsonl", "r") as f:
    for line in f:
        valset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

In [26]:
# Overview of the dataset.
print(trainset[0])

Example({'question': 'Are Smyrnium and Nymania both types of plant?', 'answer': 'yes'}) (input_keys={'question'})


In [27]:
tp = dspy.MIPROv2(
    metric=dspy.evaluate.answer_exact_match,
    auto="light",
    num_threads=16
)

In [29]:
dspy.cache.load_memory_cache("./memory_cache.pkl")

In [30]:
optimized_react = tp.compile(
    react,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,
)

2025/06/14 11:24:17 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a26508f5320d4386af80a5a922813397', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/06/14 11:24:17 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/06/14 11:24:17 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/06/14 11:24:17 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/06/14 11:24:17 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...
2025/06/14 11:24:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/ilessio/dev-agents/PARTNERS/dspy/.venv/lib/pyth

Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 15%|█▌        | 15/100 [02:00<11:24,  8.05s/it]


Bootstrapped 4 full traces after 15 examples for up to 1 rounds, amounting to 15 attempts.


Bootstrapping set 4/6


...


🏃 View run eval_minibatch_19 at: http://localhost:8080/#/experiments/322957703646966226/runs/50a39ca3340d4b519bdd548dbfaf26f0
🧪 View experiment at: http://localhost:8080/#/experiments/322957703646966226
Average Metric: 56.00 / 100 (56.0%): 100%|██████████| 100/100 [00:06<00:00, 14.43it/s]

2025/06/14 11:37:13 INFO dspy.evaluate.evaluate: Average Metric: 56 / 100 (56.0%)
2025/06/14 11:37:13 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 56.0
2025/06/14 11:37:13 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [29.0, 48.0, 50.0, 51.0, 56.0]
2025/06/14 11:37:13 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 56.0
2025/06/14 11:37:13 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/06/14 11:37:13 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/06/14 11:37:13 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 56.0!



🏃 View run eval_full_4 at: http://localhost:8080/#/experiments/322957703646966226/runs/7f6216681b4a41a3a76b7f5ea6930fa4
🧪 View experiment at: http://localhost:8080/#/experiments/322957703646966226


🏃 View run resilient-dove-503 at: http://localhost:8080/#/experiments/322957703646966226/runs/a26508f5320d4386af80a5a922813397
🧪 View experiment at: http://localhost:8080/#/experiments/322957703646966226


[Trace(trace_id=09dac65ad8184e2695bf9a8dacec8ced), Trace(trace_id=139ec564238c4dfeaad8640de18f21f8), Trace(trace_id=ec94fdb40d5045e8bb2c0789cf8e4161), Trace(trace_id=a257d20fb42f464994acbe8af500903c), Trace(trace_id=345948e9b3f74691bf18563c2a8ffa08), Trace(trace_id=560038821991493799945288b55984a8), Trace(trace_id=eee353dcf244488fadc8bb1651007f4b), Trace(trace_id=faba5ce1399e41c8baa257f0ec579c57), Trace(trace_id=22dc44225b264f75a198985c7b7a6f26), Trace(trace_id=c2ecbdee888a4d17ad2bbb53e38e351d)]

In [31]:
optimized_react.react.signature

StringSignature(question, trajectory -> next_thought, next_tool_name, next_tool_args
    instructions='Given the `question` about a specific topic, your task is to iteratively determine the necessary information needed to formulate a comprehensive `answer`. You will maintain a `trajectory` of your reasoning process and actions taken. For each step, produce the following outputs: \n\n1. `next_thought`: Reflect on the current situation and reason about the next steps you need to take to gather information.\n2. `next_tool_name`: Decide which tool to use next; you can choose to either search Wikipedia for relevant information or finish the task if you believe you have enough data to provide an answer.\n3. `next_tool_args`: Provide the arguments for the selected tool in JSON format, ensuring that the query for the Wikipedia search is clearly defined.\n\nUse the information from your `trajectory` to make informed decisions at each step, ensuring that you gather all necessary information befo

In [32]:
optimized_react.react.demos

[Example({'augmented': True, 'question': 'That Darn Cat! and Never a Dull Moment were both produced by what studio?', 'trajectory': '', 'next_thought': 'I need to find out which studio produced both "That Darn Cat!" and "Never a Dull Moment." This information is likely available on Wikipedia, so I will search for it there.', 'next_tool_name': 'search_wikipedia', 'next_tool_args': {'query': 'That Darn Cat! and Never a Dull Moment production studio'}}) (input_keys=None),
 Example({'augmented': True, 'question': 'Which of Zhucheng and Pingquan has more people?', 'trajectory': '[[ ## thought_0 ## ]]\nI need to find the population data for both Zhucheng and Pingquan to determine which one has more people. I will search for the population statistics of both locations.\n\n[[ ## tool_name_0 ## ]]\nsearch_wikipedia\n\n[[ ## tool_args_0 ## ]]\n{"query": "Zhucheng population"}\n\n[[ ## observation_0 ## ]]\n[1] «Zhucheng | Zhucheng () is a county-level city in the southeast of Shandong province, P

In [33]:
evaluator = dspy.Evaluate(
    metric=dspy.evaluate.answer_exact_match,
    devset=valset,
    display_table=True,
    display_progress=True,
    num_threads=24,
)

In [34]:
original_score = evaluator(react)
print(f"Original score: {original_score}")

Average Metric: 29.00 / 100 (29.0%): 100%|██████████| 100/100 [00:02<00:00, 48.34it/s]

2025/06/14 11:37:46 INFO dspy.evaluate.evaluate: Average Metric: 29 / 100 (29.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...","The question refers to a movie featuring ""the king of cool,"" which...",The Great Escape,✔️ [True]
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,"{'thought_0': 'I need to determine which individual, Robert Kardas...",Robert Kardashian's family is well-known for their reality TV show...,Robert Kardashian's family had their own reality TV show.,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in the film ""Shadows ...",After conducting multiple searches regarding the cast of the film ...,"There is no information available about a Russian ballerina in ""Sh...",
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,{'thought_0': 'I need to gather information about the man who appo...,"The man who appointed Amashsai was Nehemiah, a significant figure ...","The meaning of the name of the man who appointed Amashsai, Nehemia...",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'I need to find out what additional requirements or ...,To gain access to 173 countries and territories with an Austrian p...,"In addition to the Austrian passport, Austrian citizens may need t...",
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the name of the American actress...,The American actress and singer-songwriter known for her role as P...,2007,
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'The question refers to an animated film based on a ...,"The animated film ""Gnomeo & Juliet"" is based on William Shakespear...",The title characters are garden gnomes named Gnomeo and Juliet.,
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,{'thought_0': 'I need to verify the nationalities and contribution...,"Dorothy Arzner was an American film director, known for her signif...","No, both Dorothy Arzner and Richard Wallace were American film dir...",


🏃 View run eval at: http://localhost:8080/#/experiments/322957703646966226/runs/2ec04c4b98814cfe93bbc85ce783e06f
🧪 View experiment at: http://localhost:8080/#/experiments/322957703646966226
Original score: 29.0


[Trace(trace_id=e9aeca8d782d41ceb336e2871e8229e8), Trace(trace_id=534a917c5f2040c3b63e08803c610969), Trace(trace_id=389901127b844c31bcfce117434ed99d), Trace(trace_id=fe11e919b58a4654b307d7fe200fbe37), Trace(trace_id=9c7910a884ef4e8fbdc2752c6bfb22d5), Trace(trace_id=60fa5419b01d48d5b3c2738f96a0d694), Trace(trace_id=67ea24b1c2cf42aea541ae145262fe13), Trace(trace_id=55ae6892988643a494135f420f750b8d), Trace(trace_id=2dd3c2fb271e4c2494d80eed292dbc13), Trace(trace_id=cb38ab762a9b4fc9bc17fc63cb4cc72b)]

In [35]:
optimized_score = evaluator(optimized_react)
print(f"Optimized score: {optimized_score}")

Average Metric: 56.00 / 100 (56.0%): 100%|██████████| 100/100 [00:03<00:00, 33.23it/s]

2025/06/14 11:38:00 INFO dspy.evaluate.evaluate: Average Metric: 56 / 100 (56.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...","Steve McQueen, often referred to as ""the king of cool,"" starred in...",The Great Escape,✔️ [True]
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,{'thought_0': 'I need to find out which family had their own reali...,I found that Robert Kardashian's family had their own reality TV s...,Robert Kardashian,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in ""Shadows in Paradi...",The searches conducted did not yield specific information about a ...,"No specific information found regarding a Russian ballerina in ""Sh...",
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,"{'thought_0': ""I need to find out who appointed Amashsai and the m...","Amashsai was appointed by Nehemiah. The name ""Nehemiah"" means ""Yah...","""Yahweh comforts""",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'I need to find out what additional requirements are...,The search results indicate that Austrian citizens have visa-free ...,A valid passport,
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the release date of the first al...,"Katey Sagal, known for her role as Peggy Bundy, released her first...","April 19, 1994",✔️ [True]
96,What animated creatures were the title characters of the film whic...,seals,"{'thought_0': ""I need to identify the animated creatures that were...","The animated film ""Gnomeo & Juliet"" is based on William Shakespear...",Gnomeo and Juliet,
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,{'thought_0': 'I need to determine the nationalities of both Dorot...,I found that Dorothy Arzner was an American film director. Additio...,No,✔️ [True]


🏃 View run eval at: http://localhost:8080/#/experiments/322957703646966226/runs/bc98b2736965487280cb20d21dc227a3
🧪 View experiment at: http://localhost:8080/#/experiments/322957703646966226
Optimized score: 56.0


[Trace(trace_id=9f5b8384eb6f4d888ec80e7c56f7d889), Trace(trace_id=6b49b331904344289df8fa3edf4b8051), Trace(trace_id=1eb5c781857e44cba7a7371d42b7cc8c), Trace(trace_id=e902ec5a6fc241aaa93715ccd69ae4f1), Trace(trace_id=3951969c4c6f48e398cfd1d938a38e74), Trace(trace_id=7d0d1cc6d4fd46ea950ba60ec166ab60), Trace(trace_id=d8ddc222ff16447da108c898d92d03aa), Trace(trace_id=d1b1f5ac3aa54095a31c56aa60320d6d), Trace(trace_id=bcedaef6af7b4f02aa3053e58239ce09), Trace(trace_id=ce7a1a40893e472bae4c183ec98d606a)]